## Lab #3 - Build a pipeline following MLOps best practices

Now we will assume that we are working as DevOps Engineers (or Infra or Operations alternatively).

Our data scientists have provided us the workflow Cloud Formation template in YAML format for running a new model in production, and we need to build the pipeline for doing code control, versioning, audit, testing, and monitoring of the models in production.

This time, we will create the pipeline using AWS CodePipeline directly with the console.

*Note you could be using other tools for this like Jenkins or similar, or you could also use CodePipeline but build the pipelines through infra-as-code, or CLI commands, the CDK instead.*

**1. Go to the [AWS CodePipeline console](https://eu-west-1.console.aws.amazon.com/codesuite/codepipeline/pipelines)**

**2. Click on "Create pipeline"**